## Demo 

In [1]:
import demo_image_audio_text
import torch
import torch.nn as nn
import torchvision
from torchvision.models import resnet50
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
import os
from transformers import AutoTokenizer, AutoModel


LYRICS_FILES = 'lyrics'

### Load the text model and get the similarity for the text data

In [2]:
# Define the text similarity function 
def text_similarity(file_path, embedding_file, top_results):
    model_name = "bert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    lyrics_files = os.listdir(LYRICS_FILES)


    saved_embeddings = torch.load(embedding_file)
    test_lyrics = file_path
    
    with open(test_lyrics, "r") as f:
        test_lyrics = f.read()

    encoded_test = tokenizer(test_lyrics, padding=True, truncation=True, max_length=512, return_tensors="pt")

    with torch.no_grad():
        test_output = model(**encoded_test)
        test_embedding = test_output.last_hidden_state[:, 0, :]

    similarity_scores = torch.nn.functional.cosine_similarity(test_embedding, saved_embeddings)
    sorted_scores, sorted_indices = similarity_scores.sort(descending=True)

    k = top_results
    top_k_scores = sorted_scores[:k]
    top_k_indices = sorted_indices[:k]

    print("Top {} similar songs:".format(k))
    for score, index in zip(top_k_scores, top_k_indices):
        print("Song:", lyrics_files[index], "Score:", score.item())


### Load the image model and get the similarity for the image data

In [3]:
# Set up basic paths and constants
BATCH_SIZE = 32
train_data_folder = './train'

# Set up the model growth
device = torch.device('mps')

In [4]:
# Load and transform the training dataset
data_transform = {
    'train': transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]),

    'test': transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
}


# Load the data, in order to extract the features
train_dataset = torchvision.datasets.ImageFolder(root=train_data_folder, transform=data_transform['train'])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
# Create a custom Resnet50 model for our problem
class CustomResNet50(torch.nn.Module):
    def __init__(self):
        super(CustomResNet50, self).__init__()
        self.resnet = models.resnet50(pretrained=False)

    def forward(self, x):
        x = self.resnet(x)
        return x

In [6]:
# Load new model and match the weights
model = CustomResNet50()

# Load the model from the trained model
pretrained_dict = models.resnet50(pretrained=True).state_dict(torch.load('saved_models/resnet50_trained.pth'))
model_dict = model.state_dict()
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}

# Update the parameters
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)
model.to(device)
model.eval()

/opt/homebrew/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/opt/homebrew/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/opt/homebrew/lib/python3.10/site-packages/torch/nn/modules/module.py:1802: Use

CustomResNet50(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
    

In [7]:
# Get the features of all images and set them with the labels
train_features = []
train_labels = []

with torch.no_grad():
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        features = model(images)
        train_features.append(features)
        train_labels.append(labels)

train_features = torch.cat(train_features, dim=0)
train_labels = torch.cat(train_labels, dim=0)

## Get Results

### Extract the image, text data from the video

In [12]:
# Use the demo_image_text file in order to extract the image and text data from the video
demo_image_audio_text.data_creation('demo_folder', 'Duke_Dumont-Ocean_Drive.mp4')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with clang version 12.0.0
  configuration: --prefix=/Users/ktietz/demo/mc3/conda-bld/ffmpeg_1628925491858/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=arm64-apple-darwin20.0.0-clang --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
file_path = 'demo_folder/demo.txt'
embedding_path = "saved_models/embeddings.pt"
top_results = 10

text_similarity(file_path, embedding_path, top_results)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Top 10 similar songs:
Song: Bruno-Mars-That-s-What-I-Like.txt Score: 0.9069370627403259
Song: Conan-Gray-Heather.txt Score: 0.8974332809448242
Song: Post-Malone-Swae-Lee-Sunflower.csv Score: 0.8898136019706726
Song: Coldplay-X-BTS-My-Universe.txt Score: 0.8890013694763184
Song: Khalid-Young-Dumb-Broke.txt Score: 0.88477623462677
Song: Elley-Duh-MIDDLE-OF-THE-NIGHT.csv Score: 0.8831318616867065
Song: Harry-Styles-Golden.txt Score: 0.8817164897918701
Song: Marshmello-x-Jonas-Brothers-Leave-Before-You-Love-Me.csv Score: 0.8806685209274292
Song: POP-SMOKE-WHAT-YOU-KNOW-BOUT-LOVE.txt Score: 0.8787678480148315
Song: Imagine-Dragons-Bad-Liar.csv Score: 0.8710405230522156


In [14]:
import matplotlib.pyplot as plt
from PIL import Image
import random

BASE_RESULTS = 'results'

# Load 10 random images
formatted_images = []
images = random.sample(os.listdir('demo_folder/frames'), 10)
os.makedirs(BASE_RESULTS)

for image in images:
    image_path = os.path.join('demo_folder/frames', image)
    formatted_images.append(data_transform['test'](Image.open(image_path)).unsqueeze(0).to(device))

# Normalize train features (replace 'train_features' with the correct feature vector)
normalized_train_features = F.normalize(train_features, p=2, dim=1)

# Process each image
for current_image in formatted_images:
    # Normalize the image feature vector
    normalized_image_features = F.normalize(model(current_image), p=2, dim=1)

    # Cosine Similarity for the top 10 most similar frames
    cos_similarities = demo_image_audio_text.cosine_similarity(normalized_image_features, normalized_train_features)
    top_k_results = torch.topk(cos_similarities, k=10).indices

    # List of most similar images and folder for each frame
    dir_name = BASE_RESULTS + '/' +  '{}'.format(os.path.splitext(str(image_path))[0].split('_')[1])
    os.makedirs(dir_name, exist_ok=True)
    similar_image_paths = [train_dataset.samples[idx][0] for idx in top_k_results]

    for i, similar_image in enumerate(similar_image_paths):
        image = Image.open(similar_image)
        image_path = os.path.join(dir_name, os.path.basename(similar_image))

        # Plot current image and its similar image
        fig, axs = plt.subplots(1, 2, figsize=(10, 5))

        # Plot current image
        axs[0].imshow(current_image.squeeze().cpu().numpy().transpose(1, 2, 0))
        axs[0].set_title("Current Image")

        # Plot similar image
        similar_image = Image.open(similar_image)
        axs[1].imshow(similar_image)
        axs[1].set_title("Similar Image {}".format(i+1))

        # Save the plot
        plt.savefig(image_path)
        plt.close()

        print(similar_image)



Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


<PIL.PngImagePlugin.PngImageFile image mode=RGB size=640x360 at 0x3B5DEC250>
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=640x360 at 0x2F62B5720>


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


<PIL.PngImagePlugin.PngImageFile image mode=RGB size=640x360 at 0x2F638EB00>
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=640x360 at 0x2F6494070>


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

<PIL.PngImagePlugin.PngImageFile image mode=RGB size=640x360 at 0x3B0934EE0>
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=640x360 at 0x3B0DD2740>


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


<PIL.PngImagePlugin.PngImageFile image mode=RGB size=640x360 at 0x3B0D3F520>
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=640x268 at 0x3A850C640>
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=640x268 at 0x3B0AF0430>


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


<PIL.PngImagePlugin.PngImageFile image mode=RGB size=518x360 at 0x3B0EE5E70>
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=640x360 at 0x302203700>
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=640x268 at 0x3AEA60B80>
